# (PART) DATA EXPLORATION {-}
# How do you create a GWAS project directory ready for analysis?

## Explanation  
Before working with data, it's important to set up a clean and organized project directory. A consistent folder structure helps you manage scripts, datasets, and outputs across both Python and R — making your work easier to follow and share.

In this guide, we’ll create a root directory called `gwas-data-science` with four folders:

- `data/` – for datasets  
- `scripts/` – for code files  
- `images/` – for plots and charts  
- `library/` – for reusable functions  

---

**Example Folder Structure:**

```plaintext
gwas-data-science/
├── data/
├── scripts/
├── images/
└── library/
```

---


## Bash (Terminal)

You can create the entire structure using this single command:

```bash
mkdir -p gwas-data-science/{data,scripts,images,library}
cd gwas-data-science
```

## Python Code

You can also create the same folder structure in Python:


```python
import os

folders = ["data", "scripts", "images", "library"]
root = "gwas-data-science"

os.makedirs(root, exist_ok=True)
for folder in folders:
    os.makedirs(os.path.join(root, folder), exist_ok=True)

print(f"Created '{root}' project folder with subdirectories.")
```

## R Code

Here’s how to do it in R:

```R
folders <- c("data", "scripts", "images", "library")
root <- "gwas-data-science"

if (!dir.exists(root)) dir.create(root)
for (folder in folders) {
  dir.create(file.path(root, folder), showWarnings = FALSE)
}

cat("Created", root, "project folder with subdirectories.\n")
```

> ✅ A clean project directory helps you stay organized, reuse code, and avoid errors — it’s the first step toward reproducible, professional data science.



## Import libraries

```{r}
library(tidyverse)
library(rrBLUP)
library(BGLR)
library(DT)
library(SNPRelate)
library(qqman)
library(poolr)
```

# How do you prepare a public GWAS dataset for R-based analysis?

## Explanation

To support reproducible and cross-platform workflows, it's helpful to separate raw data preparation from your analysis code. In this example, we use a Bash script to download and organize a publicly available rice diversity panel dataset from [Zhao et al., 2011](http://ricediversity.org/data/sets/44kgwas/).

The script performs the following steps:

- 📥 Downloads PLINK-formatted genotype files (`.ped`, `.map`, `.fam`)
- 📦 Unzips and flattens the directory structure for easy access
- 🏷️ Renames the phenotype file for clarity (`sativa413_phenotypes.txt`)
- 📁 Moves all outputs into a consistent `data/` directory with a `sativa413_` prefix

This setup creates a clean and well-organized foundation for downstream R-based GWAS analysis.

> ✅ **Takeaway:** Using a Bash script to automate dataset download and organization keeps your R analysis environment clean, standardized, and fully reproducible. All required files are now available in the `data/` folder under the `sativa413_` prefix.

# How do you prepare a public GWAS dataset for R-based analysis?

## Explanation

To support reproducible and cross-platform workflows, it's helpful to separate raw data preparation from your analysis code. In this example, we use a Bash script to download and organize a publicly available rice diversity panel dataset from [Zhao et al., 2011](http://ricediversity.org/data/sets/44kgwas/).

The script performs the following steps:

- 📥 Downloads PLINK-formatted genotype files (`.ped`, `.map`, `.fam`)  
- 📦 Unzips and flattens the directory structure for easy access  
- 🏷️ Renames the phenotype file for clarity (`sativa413_phenotypes.txt`)  
- 📁 Moves all outputs into a consistent `data/` directory with a `sativa413_` prefix

This setup creates a clean and well-organized foundation for downstream R-based GWAS analysis.

## Bash Script

```bash
#!/bin/bash

# 🚀 Prepare rice GWAS genotype and phenotype data (PLINK format)

# --- Paths and filenames ---
ZIP_URL="http://ricediversity.org/data/sets/44kgwas/RiceDiversity.44K.MSU6.Genotypes_PLINK.zip"
ZIP_FILE="data/rice_gwas_genotypes.zip"
EXTRACT_DIR="data/RiceDiversity_44K_Genotypes_PLINK"
PHENO_URL="http://www.ricediversity.org/data/sets/44kgwas/RiceDiversity_44K_Phenotypes_34traits_PLINK.txt"
PHENO_OUT="data/sativa413_phenotypes.txt"

# --- Step 1: Ensure data folder exists ---
mkdir -p data

# --- Step 2: Download genotype zip file (if not already present) ---
if [ ! -f "$ZIP_FILE" ]; then
  echo "⬇️ Downloading genotype data..."
  wget --no-check-certificate -O "$ZIP_FILE" "$ZIP_URL"
else
  echo "✅ Genotype zip already exists: $ZIP_FILE"
fi

# --- Step 3: Unzip genotype data ---
echo "📂 Extracting genotype files..."
unzip -o "$ZIP_FILE" -d data/

# --- Step 4: Move files up and clean nested folder ---
if [ -d "$EXTRACT_DIR" ]; then
  mv "$EXTRACT_DIR"/* data/
  rm -rf "$EXTRACT_DIR"
fi
rm -rf data/__MACOSX
rm -f "$ZIP_FILE"

# --- Step 5: Download phenotype file and rename ---
echo "⬇️ Downloading phenotype file..."
wget --no-check-certificate -P data/ "$PHENO_URL"
mv data/RiceDiversity_44K_Phenotypes_34traits_PLINK.txt "$PHENO_OUT"

echo "✅ GWAS data successfully prepared in the data/ folder."
```

The Bash script is saved as:  
```bash
script/gwas_data.sh
```

To run it:
```bash
bash script/gwas_data.sh
```

## File Structure
After execution, the `data/` folder contains:

```
project/
├── script/
│   └── gwas_data.sh
└── data/
    ├── sativa413.map
    ├── sativa413.ped
    ├── sativa413.fam
    └── sativa413_phenotypes.txt
```

> ✅ **Takeaway:** Using a Bash script to automate dataset download and organization keeps your R analysis environment clean, standardized, and fully reproducible. All required files are now available in the `data/` folder under the `sativa413_` prefix.

# How do you efficiently load and store GWAS data files in R?

## Explanation

GWAS datasets can be large, especially the genotype file in `.ped` format, which stores two columns per SNP. Repeatedly reading this file is time-consuming, so it’s best to:

- Load it once using `readr::read_table()` for speed and consistency.
- Save the loaded object as a compressed `.rds` file for fast future access.
- Load supporting metadata files (`.map`, `.fam`) and phenotype data using consistent column names.

All files are stored in the `data/` directory and prefixed with `sativa413_`.

## R Code

```{r}
library(tidyverse)

# Step 1: Load genotype data using readr
ped_data <- read_table("data/sativas413.ped", col_names = FALSE, show_col_types = FALSE)

# Step 2: Save as compressed RDS file
write_rds(ped_data, file = "data/sativa413.rds")

# Step 3: Load metadata files
map_data <- read_table("data/sativas413.map", 
                       col_names = c("chr", "snp_id", "gen_dist", "bp_pos"), 
                       show_col_types = FALSE)

fam_data <- read_table("data/sativas413.fam", 
                       col_names = c("FID", "IID", "PID", "MID", "sex", "phenotype"), 
                       show_col_types = FALSE)

phenotype_data <- read_tsv("data/sativa413_phenotypes.txt", show_col_types = FALSE)
```

> ✅ **Takeaway:** For large genotype files, load once using `readr`, save as `.rds`, and always use clear column names when importing metadata and phenotype files to streamline analysis and ensure reproducibility.

# How do you inspect the structure and contents of GWAS input files in R?

## Explanation

Before analysis, it’s essential to preview each file to check:

- Row and column dimensions
- Sample ID consistency
- General format of genotype and phenotype data

This ensures everything is aligned before tidying or merging.

## R Code

```{r}
# Load genotype file (if needed)
ped_data <- read_rds("data/sativa413.rds")

# Check size of each file
dim(ped_data)         # Genotype matrix
dim(map_data)         # SNP metadata
dim(fam_data)         # Sample info
dim(phenotype_data)   # Trait info

# Preview first few rows and columns
head(ped_data[, 1:5])        # First 5 genotype columns (show alleles)
head(map_data)
head(fam_data[, 1:5])
head(phenotype_data[, 1:5])
```

> ✅ **Takeaway:** Use `dim()` and `head()` to quickly check file structure and confirm that samples and traits align before transforming or analyzing GWAS data.

# How do you tidy the genotype matrix from a `.ped` file in R?

## Explanation

PLINK `.ped` files store genotype data in a wide format, where each SNP is represented by **two columns per individual** (one for each allele). This structure is inefficient for downstream analysis, so we convert it to a **tidy format** where:

- Each row represents one sample
- Each column represents one SNP
- Alleles are combined (e.g., `"A A"`, `"A G"`, `"G G"`)

We also use SNP metadata from the `.map` file — now consistently referred to as `snp_info` — to label the columns.

## R Code

```{r}
# Load necessary library
library(tidyverse)

# Step 1: Load genotype and SNP info (if not already in memory)
ped_data <- read_rds("data/sativa413.rds")
snp_info <- read_table("data/sativas413.map", 
                       col_names = c("chr", "snp_id", "gen_dist", "bp_pos"), 
                       show_col_types = FALSE)

# Step 2: Separate sample IDs and genotype calls
sample_ids <- ped_data[, 1:2]              # FID and IID
genotype_matrix <- ped_data[, -(1:6)]      # Alleles only

# Step 3: Verify expected SNP count
n_snps <- ncol(genotype_matrix) / 2
stopifnot(n_snps == nrow(snp_info))

# Step 4: Combine each pair of allele columns into genotype strings
genotype_calls <- map_dfc(seq(1, ncol(genotype_matrix), by = 2), function(i) {
  paste(genotype_matrix[[i]], genotype_matrix[[i + 1]])
})
names(genotype_calls) <- snp_info$snp_id  # Use SNP IDs as column names

# Step 5: Combine with sample IDs
genotype_tidy <- bind_cols(sample_ids, genotype_calls)

# Step 6: Preview output
head(genotype_tidy[1:10, 1:5])  # Show FID, IID, and first 8 SNPs
glimpse(genotype_tidy[1:5, 1:10])  # Show FID, IID, and first 8 SNPs
```

> ✅ **Takeaway:** Tidying `.ped` genotype data into a clean sample-by-SNP format makes it much easier to analyze, visualize, or convert to numeric dosages. Use `snp_info` consistently as the SNP metadata reference to avoid conflicts.

# How do you recode allele strings into numeric count format for GWAS?

## Explanation

After tidying the `.ped` genotype matrix into a clean format where each SNP column contains values like `"A A"`, `"A G"`, or `"G G"`, most GWAS tools require those genotypes to be **numeric**:

> **Genotype count format:**  
> - `0` = Homozygous for major allele  
> - `1` = Heterozygous  
> - `2` = Homozygous for minor allele  
> - `NA` = Missing or uncalled genotype

To do this, we:

1. Identify the two alleles observed at each SNP  
2. Determine the **minor allele** (less frequent)  
3. Count how many copies of the **minor allele** each individual has (`0`, `1`, or `2`)

---

✅ **Clarifying the structure of the genotype matrix:**

Each pair of alleles (like `"A G"`, `"G G"`, `"T C"`) represents a **genotype** for a single SNP in a single individual.

So when you load the `.ped` file and separate it into allele pairs:

- **Each pair** = one **genotype**  
- **Each column** = one **SNP**  
- **Each row** = one **sample**

> 🧠 This distinction is important when converting genotype strings to numeric formats for GWAS.

## R Code

```{r}
# Load libraries
library(tidyverse)

# Step 1: Drop FID and IID from genotype_tidy to isolate genotype columns
geno_alleles <- genotype_tidy[, -c(1, 2)]

# Step 2: Convert allele strings to numeric minor allele counts
geno_minor_allele_count <- map_dfc(geno_alleles, function(allele_vec) {
  # Split all genotype strings (e.g., "A G") into individual alleles
  alleles <- unlist(str_split(allele_vec, " "))
  allele_counts <- table(alleles)

  # Skip SNPs that are monomorphic or malformed
  if (length(allele_counts) < 2) return(rep(NA, length(allele_vec)))

  # Identify the minor allele (less frequent)
  minor_allele <- names(sort(allele_counts))[1]

  # Count how many copies of the minor allele are in each genotype
  sapply(allele_vec, function(gt) {
    if (gt %in% c("0 0", "0 1", "1 0", "1 1", "0", "1")) return(NA)  # filter malformed
    split_alleles <- unlist(str_split(gt, " "))
    if (length(split_alleles) != 2) return(NA)
    sum(split_alleles == minor_allele)
  })
})

# Step 3: Add back sample identifiers
genotype_count <- bind_cols(genotype_tidy[, 1:2], geno_minor_allele_count)

# Step 4: Preview the cleaned matrix
glimpse(genotype_count[, 1:5])
```

> ✅ **Takeaway:** Recoding genotype strings into numeric dosages (0, 1, 2) is essential for statistical GWAS models. It standardizes input and prepares your data for PCA, association testing, or genomic prediction.

# How do you filter SNPs and samples based on missing data and minor allele frequency?# How do you filter genotype data for missingness and minor allele frequency?

## Explanation

Before running GWAS, it's important to apply basic quality control (QC) to the genotype matrix. This ensures that:

- SNPs with too many missing genotypes are excluded  
- SNPs with very low variability (low minor allele frequency) are removed  
- Samples with excessive missing data (optional) are filtered out  

These steps improve statistical power and reduce false associations.

## R Code

```{r}
# Load required libraries
library(tidyverse)

# Step 1: Remove sample columns (FID, IID)
count_only <- genotype_count[, -c(1, 2)]

# Step 2: Filter SNPs by missingness (e.g., keep SNPs with <10% missing values)
snp_missing <- colMeans(is.na(count_only))
snp_keep <- names(snp_missing[snp_missing < 0.1])
filtered_count <- count_only[, snp_keep]

# Step 3: Filter SNPs by minor allele frequency (MAF >= 0.05)
calc_maf <- function(x) {
  p <- mean(x, na.rm = TRUE) / 2
  min(p, 1 - p)
}
snp_maf <- map_dbl(filtered_count, calc_maf)
maf_keep <- names(snp_maf[snp_maf >= 0.05])
final_count <- filtered_count[, maf_keep]

# Step 4: Reattach FID and IID
filtered_geno <- bind_cols(genotype_count[, 1:2], final_count)

# Step 5: Summary of filtering
cat("Original SNPs:", ncol(count_only), "\n")
cat("After missing filter:", length(snp_keep), "\n")
cat("After MAF filter:", length(maf_keep), "\n")
```

> ✅ **Takeaway:** Apply SNP-level filters for missing data and low MAF to improve data quality. This ensures that only informative and reliable markers are used in your GWAS analysis.

# How do you impute missing genotype values before GWAS analysis?

## Explanation

Many GWAS and population structure methods (like PCA or kinship matrix computation) require complete genotype matrices. If you have filtered for missingness but still have a few `NA` values, a simple approach is to impute missing genotypes using the **mean dosage** for each SNP.

This is fast, reproducible, and good enough for visualization and many linear models.

## R Code

```{r}
# Load library
library(tidyverse)

# Step 1: Extract dosage matrix (without FID/IID)
dosage_matrix <- filtered_geno[, -c(1, 2)]

# Step 2: Impute missing values using column means
imputed_matrix <- dosage_matrix %>%
  mutate(across(everything(), ~ ifelse(is.na(.), mean(., na.rm = TRUE), .)))

# Step 3: Add back FID and IID
geno_imputed <- bind_cols(filtered_geno[, 1:2], imputed_matrix)

# Step 4: Preview
head(geno_imputed[, 1:5])
glimpse(geno_imputed[1:5, 1:10])
```

> ✅ **Takeaway:** Simple mean imputation fills missing genotype values efficiently. It’s suitable for PCA, kinship, and linear models when high accuracy isn't critical or when advanced imputation isn't available.

# How do you perform PCA on genotype data to assess population structure?

## Explanation

PCA (Principal Component Analysis) is a standard method to explore genetic diversity and detect hidden population structure in GWAS datasets. It helps:

- Control for confounding due to stratification  
- Visualize sample clusters or outliers  
- Generate covariates (`PC1`, `PC2`, etc.) for use in association models

We apply PCA on the **imputed genotype dosage matrix**, excluding identifier columns (`FID`, `IID`), and then combine the results with sample IDs.

## R Code

```{r}
# Load libraries
library(tidyverse)

# Step 1: Extract genotype matrix (exclude FID and IID)
geno_numeric <- geno_imputed[, -c(1, 2)]

# Step 2: Perform PCA using prcomp
pca_result <- prcomp(geno_numeric, center = TRUE, scale. = TRUE)

# Step 3: Combine first 5 PCs with sample IDs
pca_df <- geno_imputed[, 1:2] %>%  # FID and IID
  bind_cols(as_tibble(pca_result$x[, 1:5]))  # PC1 to PC5

# Step 4: Plot PC1 vs PC2
ggplot(pca_df, aes(x = PC1, y = PC2)) +
  geom_point(size = 2, alpha = 0.7) +
  labs(title = "PCA of Genotype Data", x = "PC1", y = "PC2") +
  theme_minimal()
```

> ✅ **Takeaway:** PCA helps uncover hidden structure in your GWAS population. Always reattach `FID` and `IID` to PCA scores so they can be merged with phenotype and genotype metadata for downstream modeling.

# How do you include PCA covariates in a GWAS model?

## Explanation

To reduce false positives caused by hidden genetic structure, it's standard in GWAS to include the top principal components (PCs) as covariates in the model. These PCs come from PCA applied to the genotype matrix and must be matched back to each sample using the same identifiers (`FID`, `IID`).

This Q&A walks through the process of combining phenotype, PCA, and genotype data, then fitting a single SNP-trait association model.

## R Code

```{r}
# Load libraries
library(tidyverse)

# Step 1: Load and align phenotype data with sample metadata (by FID)
fam_data <- read_table("data/sativas413.fam", 
                       col_names = c("FID", "IID", "PID", "MID", "sex", "phenotype"), 
                       show_col_types = FALSE)

phenotype_data <- read_tsv("data/sativa413_phenotypes.txt", show_col_types = FALSE) %>%
  rename(FID = 1)  # Sample IDs in phenotype file match fam_data$FID

sample_metadata <- left_join(fam_data, phenotype_data, by = "FID")

# Step 2: Rename PCA columns to include proper IDs
pca_df <- pca_df %>%
  rename(FID = 1, IID = 2)

# Step 3: Merge PCA data into sample metadata
sample_data <- left_join(sample_metadata, pca_df, by = c("FID", "IID"))

# Step 4: Standardize ID columns in genotype data
geno_imputed <- geno_imputed %>%
  rename(FID = 1, IID = 2)

# Step 5: Merge genotype with metadata
geno_data <- geno_imputed[, -1]  # Drop FID, keep IID and SNPs
gwas_input <- left_join(sample_data, geno_data, by = "IID")

# Step 6: Select trait and covariates
trait <- "Plant height"  # Use the column name as a string
covariates <- c("PC1", "PC2", "PC3")

# Step 7: Construct the model formula
snp_name <- names(geno_data)[2]  # Replace with desired SNP
formula_str <- paste0("`", trait, "` ~ ", snp_name, " + ", paste(covariates, collapse = " + "))
model <- lm(as.formula(formula_str), data = gwas_input)

# Step 8: View model summary
summary(model)
```

> ✅ **Takeaway:** Always ensure PCA scores include correctly labeled `FID` and `IID` so they can be merged with metadata and genotype matrices before fitting GWAS models.

# How do you interpret GWAS model results with PCA covariates?

## Explanation

Once a GWAS model is fitted using a phenotype (e.g., *Plant height*), a SNP, and population structure covariates (e.g., PC1–PC3), we interpret the results using the model summary. The key values to look for are:

- **Estimate**: The effect size of each variable  
- **Pr(>|t|)**: The *p-value*, used to determine significance  
- **R-squared**: The proportion of variation in the trait explained by the model  
- **Residuals**: The spread of errors not explained by the model  

This example tests the association between SNP `id1000007` and *Plant height*, adjusting for PC1 to PC3.

## R Model Output Summary

| Coefficient   | Estimate  | Std. Error | t value | Pr(>|t|)  | Significance |
|---------------|-----------|------------|---------|-----------|--------------|
| (Intercept)   | 115.83448 | 1.04707    | 110.63  | < 2e-16   | ***          |
| id1000007     | 2.17767   | 1.54100    | 1.413   | 0.158     |              |
| PC1           | 0.20502   | 0.02761    | 7.426   | 7.49e-13  | ***          |
| PC2           | -0.19534  | 0.04436    | -4.404  | 1.39e-05  | ***          |
| PC3           | -0.29738  | 0.07399    | -4.019  | 7.05e-05  | ***          |

**Model Fit:**

- Residual standard error: **18.63**
- Degrees of freedom: **378**
- R-squared: **0.2277**
- Adjusted R-squared: **0.2195**
- F-statistic: **27.86 on 4 and 378 DF**
- Overall p-value: **< 2.2e-16**

> ✅ **Takeaway:** This SNP is not significant (`p = 0.158`), but PCs show strong association with plant height. Controlling for population structure is essential to avoid false signals in GWAS.